<a href="https://colab.research.google.com/github/DataSpott/SGT-Analysis/blob/master/SGT_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SGT-Analysis-tool
===

Program to analyse bacterial growth-curves via the "Start Growth Time"-method.

## How to use this program

* The program is executed by clicking the Play-button, as shown in the following image, on the left-hand-side of each of the following blocks.

* The button appears when hovering over the square brackets.

 ![Play-button](https://github.com/DataSpott/SGT-Analysis/blob/master/play_button.png?raw=true)

* Due to the structure of the program it is important that each block runs after the preceding one is finished. 

* You can make a Testrun by downloading on of the two sample data-files and use the already registered parameters (see also the manual under the link provided below, for further information)

## Further detailed information

For further information please see the manual at ....







Due to limitations in the input of antibiotic concentrations the tool can only handle checkerboards with a similiar layout. To run checkerboards with different concentrations of the antibiotics please run multiple instances of the program.

## Example checkerboard-layout

![Illustration of the checkerboard-layout.](https://github.com/DataSpott/SGT-Analysis/blob/master/Checkerboard_blueprint_with_description.png?raw=true)




--------------------------------------------------------------------------------

# 1. Data-upload and program-initialization
Installation & import of necessary modules and libraries. Initializes the data-upload by the user.

In [ ]:
#@markdown ***Click the play-button to start.***

################################################################################
## Import of the necessary python-modules for computing the data
import apt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import xlrd as rd
import string
import subprocess as sub
import os
import sys

def install(package):
    sub.check_call([sys.executable, "-m", "pip", "install", package])

install('altair_saver')
install('selenium')

sub.check_call(['apt-get', 'update'])
sub.check_call(['apt-get', 'install', '-y', 'chromium-chromedriver'],
     stdout=open(os.devnull,'wb'), stderr = None) 

from scipy.optimize import curve_fit

print("--> Installation complete <--\n")


################################################################################
## Data-upload

from google.colab import files

uploaded = files.upload()
uploaded_files = {}

for fn in uploaded.keys():
    if fn.endswith(('.xls', '.xlsx')) == True:
        file_name = fn.encode('utf-8').decode(encoding = 'utf-8')
        uploaded_files[file_name] = uploaded[fn]
        print('User uploaded file "{name}" with length {length} bytes'.format(name=file_name, length=len(uploaded[file_name])))
    else:
        print('Please upload Excel-file.')


################################################################################
## Function-definitions

#Definition of the sigmoid-function:
#L = curve's maximum value; x0 = x-value of the sigmoid´s midpoint 
#(turnpoint, where the curve is linear); k = logistic growth rate or 
#steepness of the curve; n = value for y at x = 0
def sigmoid(x, L, x0, k, n):                           
    y = L / (1 + np.exp(-k * (x - x0))) + n
    return y


#defines a function to make a sigmoid-curve-fit for a given data-set in a 
#dataframe:
def curve_fit_calc(dataframe, actual_well):
    actual_dataframe_part = dataframe[dataframe['Well'] == actual_well]

    #initialize data:
    x_data = actual_dataframe_part['Time'].values
    y_data = actual_dataframe_part['Value'].values

    #initialize start-parameters for curve_fit:
    y_max = np.amax(y_data)
    y_min = np.amin(y_data)
    k = (y_max/1000)          
    half_index = int(len(x_data)/2)
    x0 = actual_dataframe_part.iloc[half_index, 1]

    #initialize parameters for the sigmoid-function (= L, x0, k, n)
    p0 = [y_max, x0, k, y_min]
        
    #curve_fit:
    popt, pcov = curve_fit(sigmoid, x_data, y_data, p0)

    #get fitted function from optimized parameters:
    fitted_y = sigmoid(x_data, *popt)
    
    fitted_values_list = [actual_well, np.array(x_data), fitted_y]
    popt_values_list = [actual_well, popt[0], popt[1], popt[2], popt[3]]
    
    return fitted_values_list, popt_values_list


#defines function to find value in a given array that is closest to a given 
#value:
def find_nearest(given_array, given_value):                                                 
    
    #converts the given list/array/... to an array:
    array = np.asarray(given_array)
    
    #finds the index of the array-value, which results in the lowest absolute 
    #value by subtraction with the searched value:
    idx = (np.abs(array - given_value)).argmin()
    
    #returns the array-value:
    return array[idx]


#defines function to calculate µ for the linear area of each growth-curve given  
#by the upper and lower boundary:
def µ_calc_by_boundaries(dataframe, actual_well, upper_boundary, lower_boundary):
            
    actual_dataframe_part = dataframe[dataframe['Well'] == actual_well]

    fitted_value_array = actual_dataframe_part['Fitted Value'].values
    
    #searches the nearest fitted values to the user-specified boundaries for the actual well:
    nearest_upper_boundary = find_nearest(fitted_value_array, upper_boundary)
    nearest_lower_boundary = find_nearest(fitted_value_array, lower_boundary)

    upper_time_index = np.where(fitted_value_array == nearest_upper_boundary)
    lower_time_index = np.where(fitted_value_array == nearest_lower_boundary)

    #searches the corresponding times to the boundary-closest fitted values:
    time_array = actual_dataframe_part['Time'].values
    upper_time = np.ndarray.item(time_array[upper_time_index])
    lower_time = np.ndarray.item(time_array[lower_time_index])

    #calculate delta_x & delta_y:
    delta_x = np.subtract(upper_time, lower_time)
    delta_y = (upper_boundary - lower_boundary)

    #check if division by 0 occured;
    if delta_x > 0 or delta_x < 0:
        
        #if FALSE calculate µ:
        µ = (delta_y/delta_x)

    #if TRUE write a comment in µ:
    else:
        µ = 'division by 0'

    return actual_well, upper_boundary, upper_time, lower_boundary, lower_time, µ


#define function to calculate n from y = m * x + n:
def n_calc(given_array):
    
    #assign variables:
    well = given_array[0]
    m = given_array[4]
    x = given_array[1]
    y = given_array[3]
    
    #check if m isn´t a string:
    if type(m) != str:
        
        #if TRUE calculate n:
        n = y - m * x
        
        #returns two lists:
        return [well, m, n], [well, x, m]
    
    #if FALSE set a note:
    else:
        
        n = 'not computable'
        
        #returns two lists:
        return [well, m, n], [well, x, m]


#define function to calculate µ at all times for all growth_curves:
def µ_calc(dataframe, actual_well):
        
    #initialize data:
    time_array = dataframe[dataframe['Well'] == actual_well]['Time'].values
    fitted_value_array = dataframe[dataframe['Well'] == actual_well]['Fitted Value'].values

    #process data:
    delta_x = np.subtract(time_array[1:], time_array[:-1])
    delta_y = np.subtract(fitted_value_array[1:], fitted_value_array[:-1])

    #calculate µ:
    µ = np.divide(delta_y, delta_x)

    return actual_well, µ, time_array[1:]


#define function to find max µ-values in array:
def find_µ_max(given_array):
    
    #process data:
    well = given_array[0]
    µ_max = np.amax(given_array[1])

    #find index of µ_max in the array:
    µ_max_index = np.where(given_array[1] == µ_max)
    
    #get to µ_max corresponding time in the array;
    µ_max_time = np.ndarray.item(given_array[2][µ_max_index])

    return well, µ_max_time, µ_max


#defines function to calculate the SGT-value for a row in a dataframe:
def SGT_calc(dataframe, actual_row, cut_off_value):
    
    #setup initial data:
    µ_max = dataframe.loc[actual_row, 'µ_max']
    y0 = dataframe.loc[actual_row, 'y0']

    #check if µ_max is no string:
    if type(µ_max) != str:
        
        #if TRUE compute SGT and add it to the dataframe:
        SGT_value = int(((cut_off_value - y0) / µ_max) / 60)
    
    #if FALSE add a comment to the dataframe:
    else:
        SGT_value = 'not computable'
    
    return SGT_value


#defines function to calculate the log-level-reduction for a row of a dataframe:
def log_calc(dataframe, actual_row, first_well, last_well, antibiotic_rows_conc, antibiotic_columns_conc, actual_board, board_count):
    
    #get data:
    actual_well_name = str(dataframe.iat[actual_row, 0])
    actual_well_letter = actual_well_name[0]
    actual_well_number = int(actual_well_name[1:])
    
    SGT_value = dataframe.loc[actual_row, 'SGT']
    
    actual_well_dataframe_part = dataframe[dataframe['Well'] == actual_well_name]
    first_well_dataframe_part = dataframe[dataframe['Well'] == first_well]
    
    #check if well is in the limits of the checkerboard:
    if  actual_well_letter >= first_well[0] and  actual_well_letter <= last_well[0]:
        if actual_well_number >= int(first_well[1:]) and actual_well_number <= int(last_well[1:]):
            
            #if TRUE check if the SGT-value is no string:
            if type(SGT_value) != str:
                
                #if TRUE calculate data:
                SGT_subtracted_value = int(float(actual_well_dataframe_part['SGT'].values[0]) 
                                      - float(first_well_dataframe_part['SGT'].values[0]))
                log_level_reduction = round((SGT_subtracted_value / log_time), 1)
            
                #determine antibiotic-concentrations:
                first_antibiotic_index = (ord(actual_well_letter) - ord(first_well[0]))
                first_antibiotic_concentration = float(antibiotic_rows_conc[first_antibiotic_index])
                second_antibiotic_index = int(actual_well_number - ((12 / board_count) * actual_board)) - 1
                second_antibiotic_concentration = float(antibiotic_columns_conc[second_antibiotic_index])

                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]
            
            #if FALSE set note and determine antibiotic-concentrations:    
            else:
                
                SGT_subtracted_value = 'not computable'
                log_level_reduction = 'not computable'
                first_antibiotic_index = (ord(actual_well_letter) - ord(first_well[0]))
                first_antibiotic_concentration = float(antibiotic_rows_conc[first_antibiotic_index])
                second_antibiotic_index = int(actual_well_number - ((12 / board_count) * actual_board)) - 1
                second_antibiotic_concentration = float(antibiotic_columns_conc[second_antibiotic_index])
                
                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]

        else:
            
            return 'not on checkerboard'

    #else check if actual well is over or under the checkerboard:
    elif actual_well_letter <= first_well[0] or  actual_well_letter >= last_well[0]:
        if actual_well_number >= int(first_well[1:]) and actual_well_number <= int(last_well[1:]):
            
            #if TRUE check if the SGT-value is no string:
            if type(SGT_value) != str:
                
                #calculate data:
                SGT_subtracted_value = int(float(actual_well_dataframe_part['SGT'].values[0]) 
                                  - float(first_well_dataframe_part['SGT'].values[0]))
                log_level_reduction = round((SGT_subtracted_value / log_time), 1)
                
                #set string for antibiotic-concentrations:
                first_antibiotic_concentration = '-'
                second_antibiotic_concentration = '-'

                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]
            
            #if FALSE set note and set string for antibiotic-concentrations:
            else:
                SGT_subtracted_value = 'not computable'
                log_level_reduction = 'not computable'
                first_antibiotic_concentration = '-'
                second_antibiotic_concentration = '-'
                
                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]
        else:
            
            return 'not on checkerboard'

    #else check if actual well is left or right of the checkerboard:
    elif actual_well_number <= int(first_well[1:]) or actual_well_number >= int(last_well[1:]):
        if  actual_well_letter >= first_well[0] and  actual_well_letter <= last_well[0]:
            
            #if TRUE check if the SGT-value is no string:
            if type(SGT_value) != str:
                
                #calculate data:
                SGT_subtracted_value = int(float(actual_well_dataframe_part['SGT'].values[0]) 
                                  - float(first_well_dataframe_part['SGT'].values[0]))
                log_level_reduction = round((SGT_subtracted_value / log_time), 1)
                
                #set string for antibiotic-concentrations:
                first_antibiotic_concentration = '-'
                second_antibiotic_concentration = '-'

                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]

            #if FALSE set note and set string for antibiotic-concentrations:
            else:
                SGT_subtracted_value = 'not computable'
                log_level_reduction = 'not computable'
                first_antibiotic_concentration = '-'
                second_antibiotic_concentration = '-'
                
                return [actual_well_name, SGT_subtracted_value, 
                    log_level_reduction, first_antibiotic_concentration, 
                    second_antibiotic_concentration]
        else:
            
            return 'not on checkerboard'

    else:
            
        return 'not on checkerboard'


#defines function to apply function 'log_calc' to every checkerboard:
def board_iteration(dataframe, antibiotic_rows_conc_list, antibiotic_columns_conc_list, checkerboard_list, actual_board, board_count):

    #get row_count of the dataframe:
    row_count = dataframe.shape[0]

    #set first and last well of the checkerboard:
    first_well = str(checkerboard_list[actual_board][1]).strip()
    last_well = str(checkerboard_list[actual_board][2]).strip()

    #calls function 'log_calc' for every row in 'row_count':
    calculated_values_for_board = (list(log_calc(dataframe, rows, first_well, last_well, antibiotic_rows_conc_list, antibiotic_columns_conc_list, actual_board, board_count) for rows in range(row_count)))
    
    #removes list-entries from rows, whose well wasn´t at the checkerboard:
    calculated_values_for_board = list(filter(('not on checkerboard').__ne__, calculated_values_for_board))
    
    return calculated_values_for_board


#defines function, which gets the first/second antibiotic-concentration for 
#every row of a dataframe if the well is inside a specific boundary of the
#checkerboard & accomplishs the log-level-reduction:
def BBC_well_determination(dataframe, actual_row, first_well, last_well):
    
    well_name = str(dataframe.at[actual_row, 'Well'])
    log_level_reduction = dataframe[dataframe['Well'] == well_name]['log-level-reduction'].values[0]

    first_antibiotic_concentrations = []
    second_antibiotic_concentrations = []

    #check if well is within the limits of the checkerboard and accomplishs the 
    #log-level-reduction:
    if well_name[1:] == first_well[1:]:
        if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
            if log_level_reduction >= 3.0:

                #get concentration of the first antibiotic in this well:
                first_antibiotic_concentrations.append(dataframe.loc[actual_row, first_antibiotic_column])

    #check if well is within the limits of the checkerboard and accomplishs the
    #log-level-reduction:
    if well_name[0] == first_well[0]:
        if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:
            if data[data['Well'] == well_name]['log-level-reduction'].values[0] >= 3.0:
                
                #get concentration of the second antibiotic in this well:
                second_antibiotic_concentrations.append(data.loc[actual_row, second_antibiotic_column])
    
    return first_antibiotic_concentrations, second_antibiotic_concentrations


#defines function, which calls the function 'BBC_well_determination' for every
#checkerboard:
def BBC_determination_per_board(dataframe, checkerboard_list, actual_board):
    
    #get row_count of the dataframe:
    row_count = dataframe.shape[0]

    #get first & last well of the checkerboard:
    first_well = str(checkerboard_list[actual_board][1]).strip()
    last_well = str(checkerboard_list[actual_board][2]).strip()

    #calls function 'BBC_well_determination' for every row in 'row_count' &
    #splits output into two lists:
    first_antibiotic_concentrations, second_antibiotic_concentrations = map(list, zip(*list(BBC_well_determination(data, rows, first_well, last_well) for rows in range(row_count))))
    
    #delete all empty, nested lists & join the remaining nested lists to one
    #list:
    first_antibiotic_concentrations = [inner for outer in (element for element in first_antibiotic_concentrations if element != []) for inner in outer]
    second_antibiotic_concentrations = [inner for outer in (element for element in second_antibiotic_concentrations if element != []) for inner in outer]

    return first_antibiotic_concentrations, second_antibiotic_concentrations


#defines function, which calculates the FICI-value for an given well:
def fici_calc(dataframe, actual_row, first_well, last_well, first_antibiotic_bbc_on_actual_board, second_antibiotic_bbc_on_actual_board, name_of_first_antibiotic_column, name_of_second_antibiotic_column):
    
    #get name of the actual well:
    well_name = str(dataframe.at[actual_row, 'Well'])
    well_letter = well_name[0]
    well_number = int(well_name[1:])

    log_level_reduction = dataframe[dataframe['Well'] == well_name]['log-level-reduction'].values[0]

    #check if well is in the limits of the checkerboarda and the log-level-
    #reduction is >= 3:
    if well_letter > first_well[0] and well_letter <= last_well[0]:
        if well_number > int(first_well[1:]) and well_number <= int(last_well[1:]):
            if type(log_level_reduction) != str:
                if log_level_reduction >= 3.0:
                    
                    #if TRUE gets the actual concentrations of the first &
                    #second antibiotic in the well:
                    actual_first_antibiotic_concentration = dataframe[dataframe['Well'] == well_name][name_of_first_antibiotic_column].values[0]
                    actual_second_antibiotic_concentration = dataframe[dataframe['Well'] == well_name][name_of_second_antibiotic_column].values[0]
                    
                    #calculates the FICI-value:
                    well_fici_value = ((actual_first_antibiotic_concentration/first_antibiotic_bbc_on_actual_board) + (actual_second_antibiotic_concentration/second_antibiotic_bbc_on_actual_board))

                    return well_name, well_fici_value


#defines function, which calls function 'fici_calc' for every checkerboard: 
def fici_values_per_board_calc(dataframe, checkerboard_list, actual_board, first_antibiotic_first_bbc_value_list, second_antibiotic_first_bbc_value_list, name_of_first_antibiotic_column, name_of_second_antibiotic_column):
    
    #get row_count of the dataframe:
    row_count = dataframe.shape[0]

    #get first & last well of the checkerboard:
    first_well = str(checkerboard_list[actual_board][1]).strip()
    last_well = str(checkerboard_list[actual_board][2]).strip()
    
    #get the first BBC of every antibiotic for the checkerboard:
    first_antibiotic_bbc_on_actual_board = first_antibiotic_first_bbc_value_list[actual_board]
    second_antibiotic_bbc_on_actual_board = second_antibiotic_first_bbc_value_list[actual_board]
    
    #calls function 'fici_calc' for every row in 'row_count':
    fici_values_per_well = list(fici_calc(data, rows, first_well, last_well,
                                           first_antibiotic_bbc_on_actual_board,
                                           second_antibiotic_bbc_on_actual_board,
                                           name_of_first_antibiotic_column,
                                           name_of_second_antibiotic_column
                                           ) for rows in range(row_count))

    #deletes all 'None'-entries in 'fici_values':
    fici_values_per_well = [element for element in fici_values_per_well if element]    

    #splits 'fici_values_per_well' into well-name & FICI-value:
    well_name = list(elements[0] for elements in fici_values_per_well)
    fici_value = list(elements[1] for elements in fici_values_per_well)

    #appends 'well_name' & 'fici_value' to a new list:
    fici_values_per_board = [well_name, fici_value]
    
    return fici_values_per_board


#defines function which seeks the first letter in alphabetical order and the 
#smallest number in an given array of well-names:
def well_naming(given_array):
    
    #sets the letter and number of the first well in the list:
    well_letter = given_array[0][0]
    well_number = given_array[0][1:]
    
    #tests for all wells of the list if the letter and number are smaller and 
    #if true sets them as new minimal value:
    for wells in range(len(given_array)):
        if given_array[wells][0] < well_letter:
            well_letter = given_array[wells][0]
        if given_array[wells][1:] < well_number:
            well_number = given_array[wells][1:]
    
    #returns the minimal letter and number of all wells:
    return well_letter, well_number


#defines function which finds the first well in each row of a checkerboard,
#with an log-level-reduction >= 3:
def first_in_row(well_list, actual_letter, first_well_in_row_list, well_min_letter_and_number):

    #checks if the last well in the list contains the minimal number in
    #it´s name:
    end_well_number = well_min_letter_and_number[1]
    
    if len(first_well_in_row_list) > 0:
        
        if first_well_in_row_list[-1][1:] == end_well_number:   
            
            #if TRUE the loop is interrupted
            return
        
        else:

            #takes the actual letter and searches the well-list for all wells,
            #whose name contains the letter and therefore are in one row:
            well_selection = [well_name for well_name in well_list if actual_letter in well_name]

            #checks if wells whose names contain the letter were found:
            if len(well_selection) > 0:
                
                #gets the "smallest" well_name of the selected ones (the first 
                #for this row) and appends it to the array:
                first_well_in_row_list.append(min(well_selection))
                
                return first_well_in_row_list

    else:

        #takes the actual letter and searches the well-list for all wells,
        #whose name contains the letter and therefore are in one row:
        well_selection = [well_name for well_name in well_list if actual_letter in well_name]

        #checks if wells whose names contain the letter were found:
        if len(well_selection) > 0:
            
            #gets the "smallest" well_name of the selected ones (the first 
            #for this row) and appends it to the array:
            first_well_in_row_list.append(min(well_selection))
            
            return first_well_in_row_list


#defines function which finds the first well in each column of a checkerboard,
#with an log-level-reduction >= 3:
def first_in_column(well_list, actual_number, first_well_in_column_list, well_min_letter_and_number):
    
    #checks if the last well in the list contains the minimal letter in
    #it´s name:
    end_well_letter = well_min_letter_and_number[0]
    
    if len(first_well_in_column_list) > 0:
        
        if first_well_in_column_list[-1][0] == end_well_letter:   
                
            #if TRUE the loop is interrupted
            return
        
        else:

            #takes the actual number and searches the well-list for all wells
            #whose name contains the number and therefore are in one column:
            well_selection = [well_name for well_name in well_list if actual_number in well_name]

            #checks if wells which names contain the number were found:
            if len(well_selection) > 0:
                
                #gets the "smallest" of the selected wells (the first for this
                #column) and appends it to the array:
                first_well = min(well_selection)
                first_well_in_column_list.append(first_well)
                #print(first_well_in_column_list)
                return first_well_in_column_list
    
    else:

            #takes the actual number and searches the well-list for all wells
            #whose name contains the number and therefore are in one column:
            well_selection = [well_name for well_name in well_list if actual_number in well_name]

            #checks if wells which names contain the number were found:
            if len(well_selection) > 0:
                
                #gets the "smallest" of the selected wells (the first for this
                #column) and appends it to the array:
                first_well = min(well_selection)
                first_well_in_column_list.append(first_well)
                #print(first_well_in_column_list)
                return first_well_in_column_list


#defines function which identifies the first wells in all rows & columns on a
#checkerboard with an log-level-reduction >= 3 by calling the functions
#'well_naming', 'first_in_row' & 'first_in_column':
def first_well_identifier(checkerboard_list, actual_board, fici_values_list):
    
    first_well_in_row = []
    first_well_in_column = []

    #gets the wells with an log-level-reduction >= 3:
    wells = fici_values[actual_board][0]

    #appends letter and number of each well in 'wells' to a list, deletes
    #duplicates & sorts the list:
    well_letters = sorted(list(dict.fromkeys(list(wells[0] for wells in wells))))
    well_numbers = sorted(list(dict.fromkeys(list(wells[1:] for wells in wells))))

    #searches the minimal letter and number of this wells:
    well_min_letter_and_number = well_naming(wells)

    #calls functions 'first_in_row' & 'first_in_column' for all well-letters
    #/numbers:
    first_well_in_row = list(first_in_row(wells, letters, first_well_in_row, well_min_letter_and_number) for letters in well_letters)
    first_well_in_column = list(first_in_column(wells, numbers, first_well_in_column, well_min_letter_and_number) for numbers in well_numbers)

    #deletes 'None'-elements & duplicates in the lists:
    first_well_in_row = list(element for element in first_well_in_row if element != None)
    first_well_in_row = list(dict.fromkeys(inner for outer in first_well_in_row for inner in outer))
    
    first_well_in_column = list(element for element in first_well_in_column if element != None)
    first_well_in_column = list(dict.fromkeys(inner for outer in first_well_in_column for inner in outer))

    #merges both lists & deletes duplicates:
    first_well_in_row = list(dict.fromkeys(first_well_in_row + first_well_in_column))

    return first_well_in_row


#defines function which pairs the wells of the 'first_wells_in_rows_and_columns
#_list' with their corresponding FICI-value:
def FICI_finder(fici_values_list, first_wells_in_rows_and_columns_list, actual_board, actual_element):
    
    #searches the index of the actual well in the fici-values-array:
    well_index = fici_values_list[actual_board][0].index(first_wells_in_rows_and_columns_list[actual_board][actual_element])

    #appends the fici_value which corresponds to this index to the list:
    corresponding_fici_value = (fici_values_list[actual_board][1][well_index])

    #gets the actual well
    actual_well = first_wells_in_rows_and_columns_list[actual_board][actual_element]
    
    return actual_well, corresponding_fici_value


#defines function which calls function 'FICI_finder' for every checkerboard:
def FICI_finder_per_board(actual_board, fici_values_list, first_wells_in_rows_and_columns_list):
    
    #calls function 'FICI_finder' for every element in 'first_wells_in_rows_and
    #_columns_list' for the actual checkerboard:
    corresponding_fici_value_with_well = list(FICI_finder(fici_values_list, first_wells_in_rows_and_columns_list, actual_board, elements) for elements in range(len(first_wells_in_rows_and_columns_list[actual_board])))
    
    return corresponding_fici_value_with_well


#defines function which appends the FICI-values from a list of Well-name- &
#FICI-value-pairs to a new list:
def append_FICI_values_to_board(fici_values_with_well_list, actual_board):
    
    board_fici_values = list(element[1] for element in fici_values_with_well_list[actual_board])

    return board_fici_values


#defines function which calculates the y-values for the tangent of each
#growth-curve:
def tangent_value_calc(dataframe_one, dataframe_two, actual_well, time_values):
    
    actual_well_dataframe_part = dataframe_one[dataframe_one['Well'] == actual_well]

    #get µ, n the index of µ_max and the to µ-max corresponding fitted value:
    µ = actual_well_dataframe_part['µ_max'].values[0]
    n = actual_well_dataframe_part['y0'].values[0]
    µ_max_index = dataframe_two.loc[(dataframe_two['Well'] == actual_well) & (pd.notna(dataframe_two['µ_max'])), 'µ_max'].index.values[0]
    dataframe_two.loc[µ_max_index, 'µ_max corresponding Value'] =dataframe_two.loc[µ_max_index, 'Fitted Value']

    #calculate the tangent & append data to list if µ isn´t a string:
    if type(µ) != str:  
            
            calc_tangent_values = [[actual_well, time,((time * µ) + n)] for time in time_values]

    else :
        
        calc_tangent_values = 'not computable'

    return  calc_tangent_values


################################################################################
## End-condition

first_block_done = True
print('--> Initialization complete <--')

# 2. Enter Parameters
Enter the layout-specifications for your checkerboard, the log-growth-time and information to the used antibiotics.

In [ ]:
################################################################################
## Initialize parameters:
    
#@markdown # Plate-layout

#@markdown Choose your well-plate-format:
well_plate_format = "96" #@param ["24", "48", "96", "384"]
well_plate_format = int(well_plate_format)


#@markdown Enter the number of checkerboards on your well-plate:
board_count = "2" #@param {type:"string"}
board_count = int(board_count)

#@markdown Enter the first well of each checkerboard, separated by comma:
first_wells = 'A01, A07' #@param {type:"string"}
first_wells = first_wells.split(',')

#@markdown Enter the last well of each checkerboard, separated by comma:
last_wells = 'G06, G12' #@param {type:"string"}
last_wells = last_wells.split(',')

checkerboards = []

#for-loop over all checkerboards in the board-count:
for board in range(board_count):
    checkerboards.append([board, first_wells[board], last_wells[board]])

#@markdown ___
#@markdown # Log-time

#@markdown Enter the time in minutes that the investigated bacteria need to grow one log-level:
log_time = '70' #@param {type:"string"}
log_time = int(log_time)

#@markdown ___
#@markdown # Antibiotic-information

#@markdown Enter the name of the first antibiotic:
antibiotic_one_name = 'Nitroxolin' #@param {type:"string"}

#@markdown Enter the name of the second antibiotic:
antibiotic_two_name = 'Dalbavancin' #@param {type:"string"}

#@markdown Enter concentrations in µg/ml of the first antibiotic in ascending order & separated by comma:
antibiotic_one_conc = '0, 4, 8, 16, 32, 64, 128' #@param {type:"string"}
antibiotic_one_conc = [float(s) for s in antibiotic_one_conc.split(',')]
antibiotic_one_conc = sorted(antibiotic_one_conc)

#@markdown Enter concentrations in µg/ml of the second antibiotic in ascending order & separated by comma:
antibiotic_two_conc = '0, 0.5, 1, 2, 4, 8' #@param {type:"string"}
antibiotic_two_conc = [float(s) for s in antibiotic_two_conc.split(',')]
antibiotic_two_conc = sorted(antibiotic_two_conc)

#@markdown ___
#@markdown ***Click the play-button to start.***
    
    
################################################################################
## Start-condition:

try:
    first_block_done
except NameError:
    first_block_done_exists = False
else:
    first_block_done_exists = True

if first_block_done_exists == True:


################################################################################
## Check if parameters fit

    #get number of given concentrations for each antibiotic:
    antibiotic_one_conc_count = len(antibiotic_one_conc)
    antibiotic_two_conc_count = len(antibiotic_two_conc)

    letter_list = []
    possible_wells_per_board = []

    for boards in range(len(checkerboards)):
        
        import string
        
        #get first & last well of the checkerboard:
        first_well= first_wells[boards].strip()
        last_well = last_wells[boards].strip()
        
        #generate list of all letters used in well-names on the checkerboard:
        first_well_character_index = int(ord(first_well[0]) - ord('A'))
        last_well_character_index = int(ord(last_well[0]) - ord('A')) + 1
        letter_list.append(list(string.ascii_uppercase[first_well_character_index:last_well_character_index]))
        
        #generate list of all numbers used in well-names on the checkerboard:
        first_well_number = int(first_well[1:])
        last_well_number = int(last_well[1:])
        step_count = last_well_number - first_well_number + 1
        number_list = np.linspace(first_well_number, last_well_number, step_count)
        
        board_wells = []
        
        #generate list of all well-names for the checkerboard:
        for letter in letter_list[boards]:
            for number in number_list:
                if number < 10:
                    possible_well = str(letter) + '0' + str(int(number))
                    board_wells.append(possible_well)
                
                else:
                    possible_well = str(letter) + str(int(number))
                    board_wells.append(possible_well)
        
        possible_wells_per_board.append(board_wells)

    possible_well_list = []

    #combine all wellsof all checkerboards in one list:
    for sublists in range(len(possible_wells_per_board)):
        possible_well_list.extend(possible_wells_per_board[sublists])

    #sort wells in combined list:
    possible_well_list = sorted(possible_well_list)

    well_letter_list = []
    well_number_list = []

    for boards in range(len(checkerboards)):
        
        #get first & last well of the checkerboard:
        first_well = checkerboards[boards][1].strip()
        last_well = checkerboards[boards][2].strip()
        
        checkerboard_well_letter_list = []
        checkerboard_well_number_list = []
        
        #check if actual well is within boundaries of the checkerboard:
        for wells in possible_well_list:
            if wells[0] >= first_well[0] and wells[0] <= last_well[0]:
                if wells[1:] >= first_well[1:] and wells[1:] <= last_well[1:]:
                    
                    #get letter & number of the actual well:
                    well_letter = wells[0]
                    well_number = wells[1:]
                    checkerboard_well_letter_list.append(well_letter)
                    checkerboard_well_number_list.append(well_number)
        
        #delete duplicates in the lists with the well-name letters & numbers:
        checkerboard_well_letter_list = list(dict.fromkeys(checkerboard_well_letter_list))
        checkerboard_well_number_list = list(dict.fromkeys(checkerboard_well_number_list))

        well_letter_list.append(checkerboard_well_letter_list)
        well_number_list.append(checkerboard_well_number_list)

        #check if for both antibiotics the same amount of concentrations was given:
        if antibiotic_one_conc_count != antibiotic_two_conc_count:
            
            #if FALSE assign that antibiotic one rises in the rows or columns & antibiotic two in the remaining:
            if antibiotic_one_conc_count == len(well_letter_list[boards]):
                antibiotic_rows_name = antibiotic_one_name
                antibiotic_rows_conc = antibiotic_one_conc
                
                if antibiotic_two_conc_count == len(well_number_list[boards]):
                    antibiotic_columns_name = antibiotic_two_name
                    antibiotic_columns_conc = antibiotic_two_conc
                    
                    second_block_first_half_done = True

                else:
                    print('Number of given concentrations for antibiotic two doesnt´t match the number columns of the checkerboard-layout, when assigning antibiotic one to the rows. Please check your parameters')
                    
                    second_block_first_half_done = False
                
            elif antibiotic_one_conc_count == len(well_number_list[boards]):
                antibiotic_columns_name = antibiotic_one_name
                antibiotic_columns_conc = antibiotic_one_conc
                
                if antibiotic_two_conc_count == len(well_letter_list[boards]):
                    antibiotic_rows_name = antibiotic_two_name
                    antibiotic_rows_conc = antibiotic_two_conc
                    
                    second_block_first_half_done = True

                else:
                    print('Number of given concentrations for antibiotic two doesnt´t match the number of rows of the checkerboard-layout, when assigning antibiotic one to the columns. Please check your parameters.')
                    
                    second_block_first_half_done = False

            elif antibiotic_one_conc_count != len(well_letter_list[boards]) and antibiotic_one_conc_count != len(well_number_list[boards]):
                print('Number of given concentrations for antibiotic one doesn´t match neither the number of rows nor the number of columns of checkerboard-layout. Please check your parameters.')
                
                second_block_first_half_done = False

        #if TRUE assign that antibiotic one rises in the rows & antibiotic two in the columns:
        else:
            
            if antibiotic_one_conc_count == len(well_letter_list[boards]):
                antibiotic_rows_name = antibiotic_one_name
                antibiotic_rows_conc = antibiotic_one_conc
                
                if antibiotic_two_conc_count == len(well_number_list[boards]):
                    antibiotic_columns_name = antibiotic_two_name
                    antibiotic_columns_conc = antibiotic_two_conc

                    second_block_first_half_done = True

                else:
                    print('Number of given concentrations of both antibiotics doesn´t match with the given checkerboard-layout. Please check that the number of rows and columns match with the number of given concentrations for each antibiotic.')
                    
                    second_block_first_half_done = False

            else:
                print('Number of given concentrations of both antibiotics doesn´t match with the given checkerboard-layout. Please check that the number of rows and columns match with the number of given concentrations for each antibiotic.')
                
                second_block_first_half_done = False


################################################################################
## Plot checkerboard-layouts on plate

    if second_block_first_half_done == True:

        if well_plate_format == 24:
            row_count = 4
            column_count = 6
        elif well_plate_format == 48:
            row_count = 6
            column_count = 8 
        elif well_plate_format == 96:
            row_count = 8
            column_count = 12 
        elif well_plate_format == 384:
            row_count = 16
            column_count = 24

        #generate list of letters and numbers for possible wells depending on the
        #checkerboard-format:
        letter_list = list(string.ascii_uppercase[:row_count])
        number_list = np.linspace(1, column_count, column_count)

        possible_wells = []

        #generate possible wells for the plate-format:
        for letter in letter_list:
            for number in number_list:
                if number < 10:
                    possible_well = str(letter) + '0' + str(int(number))
                    possible_wells.append(possible_well)
                else:
                    possible_well = str(letter) + str(int(number))
                    possible_wells.append(possible_well)

        well_letter_indexes_for_well_plot = []
        well_number_for_well_plot = []
        well_color_for_well_plot = []

        #reverse the list of possible well-letters:
        letter_list_reverse = sorted(letter_list, reverse = True)

        for wells in possible_wells:
            
            #get letter & number of the actual well:
            actual_well_letter = wells[0]
            actual_well_number = int(wells[1:])
            
            #get the index of the letter in a reversed list of possible letters:
            actual_well_letter_index = (letter_list_reverse.index(actual_well_letter) + 1)
            
            well_letter_indexes_for_well_plot.append(actual_well_letter_index)
            well_number_for_well_plot.append(actual_well_number)
            well_color_for_well_plot.append('grey')

        well_plot_list = [possible_wells, well_number_for_well_plot, well_letter_indexes_for_well_plot, well_color_for_well_plot]

        #creates dataframe 'data_well_plot':
        data_well_plot = pd.DataFrame(well_plot_list)
        data_well_plot = data_well_plot.transpose()
        well_plot_columns = list(data_well_plot.columns)
        data_well_plot = data_well_plot.rename(columns={well_plot_columns[0] : 'Well', well_plot_columns[1] : 'x', well_plot_columns[2] : 'y', well_plot_columns[3] : 'color'})

        #initialize list of colors:
        colors = ['blue', 'red', 'yellow', 'orange', 'green', 'pink', 'teal', 'purple']

        for boards in range(len(checkerboards)):
            
            #get first and last well for the actual checkerboard:
            first_well= first_wells[boards].strip()
            last_well = last_wells[boards].strip()
            
            for wells in data_well_plot['Well']:
                
                #check if actual well is within the boundaries of the checkerboard:
                if wells[0] >= first_well[0] and wells[0] <= last_well[0]:
                    if wells[1:] >= first_well[1:] and wells[1:] <= last_well[1:]:
                        
                        #if TRUE get the index of the actual well in the dataframe and 
                        #set it´s color corresponding to the checkerboard:
                        actual_well_index = data_well_plot[data_well_plot['Well'] == wells].index.values[0]
                        data_well_plot.loc[actual_well_index, 'color'] = colors[boards]

        #add empty space at index 0 of the reversed letter-list for plotting:
        letter_list_reverse_extended = ['']
        letter_list_reverse_extended.extend(letter_list_reverse)

        #add empty space at index 0 of list with column-numbers:
        column_count_list = np.linspace(1, column_count, column_count, dtype = int)
        column_count_list_extended = ['']
        column_count_list_extended.extend(column_count_list)

        #set plot size, axis ranges and plot:
        plt.figure(figsize= (10, 10))
        plt.xticks(np.arange(len(column_count_list_extended)), column_count_list_extended)
        plt.yticks(np.arange(len(letter_list_reverse_extended)), letter_list_reverse_extended)
        plt.rc('xtick', labelsize=15)
        plt.rc('ytick', labelsize=15)
        plt.xlim(0, column_count + 1)
        plt.ylim(0, row_count + 1)
        plt.scatter(data_well_plot['x'], data_well_plot['y'], s=350, c= data_well_plot['color'], alpha=1.0)


################################################################################
## End-condition:
        
        second_block_done = True


################################################################################
## Start-condition not met:

else:
    print('-> Please run the previous block first and upload an Excel-file. <-')        

# 3. Plot fitted sigmoid curves per Well
Computes a sigmoid-curve-fit for the uploaded data and plot it, so that the user can decide if he want to set manually a linear area for the SGT-calculation.

In [ ]:
#@markdown ***Click the play-button to start.***


################################################################################
## Start-condition:

try:
    second_block_done
except NameError:
    second_block_done_exists = False
else:
    second_block_done_exists = True

if second_block_done_exists == True:
    

################################################################################
## Conversion to .csv-file

    for keys in uploaded_files.keys():
        file_name = keys

    #convert the Excel-file into a .csv-file (.csv-file = Pandas dataframe):
    data_xls = pd.read_excel(file_name, sheet_name=0)


################################################################################
## Parsing & creation of dataframe

    if pd.notna(data_xls.iloc[0, 0]):
        
        #initialize list with A-Z and a variable count of numbers:
        letter_list = list(string.ascii_uppercase)
        number_list = np.linspace(1, 50, 50)

        #combines the letters with the numbers to get a list of possible well-names:
        #creates first empty main-array:
        possible_wells = []

        #primary for-loop over all letters:
        for letter in letter_list:
        
            #secondary for-loop over all numbers:
            for number in number_list:
            
                #combines the acutal letter and number and appends it to main-array:
                possible_well = letter + str(int(number))
                possible_wells.append(possible_well)

        #variable that references the name of the first column of the dataframe 'data_xls':
        first_column_name = data_xls.columns[0]

        #creates second empty main-array:
        well_index_list = []

        #for-loop over all possible wells:
        for well in possible_wells:
        
          #searches the actual possible well in the dataframe 'data_xls' and gets the index:
          index = data_xls.loc[data_xls[first_column_name] == well, [first_column_name]].index.tolist()
        
          #checks if the resulting index is not empty (so that the well was found in the dataframe):
          if len(index) > 0:
            
              #if TRUE the well-name and index are attached to the second main-array
              well_index_list.append([well, index])

        #creates main- and helper-array:
        time_list = []
        get_time_list = []

        #extract the time-data from the dataframe in the helper-array: 
        get_time_list.extend(data_xls.iloc[(int(well_index_list[0][1][0]) + 1), 1:])

        #append helper-array to third main-array:
        time_list.append(['Well', get_time_list])

        #creates fourth empty main-array:
        well_values = []

        #for-loop over all wells found in the dataframe:
        for well in range(len(well_index_list)):
        
            #gets the name of the actual well:
            well_name = well_index_list[well][0]
        
            #checks if the number in the well_name is < 10:
            if int(well_name[1:]) < 10:
                
                #if TRUE a 0 is added after the letter:
                well_name = well_name[0] + '0' + well_name[1:]
        
            #appends well-name and the corresponding values to the fourth 
            #main-array:
            well_values.append([well_name, list(data_xls.iloc[(int(well_index_list[well][1][0]) + 3), 1:].values)])

        #extends the list with the time-data with the value-data of the wells:
        time_list.extend(well_values)

        #creates empt dictionary:
        raw_data_dict = {}

        #for-loop over all elements in the extended time_list:
        for times in range(len(time_list)):
        
            #sets the well-name (or in case of the time-data the string 'Well') 
            #as keys for their values in the dicitionary:
            raw_data_dict[str(time_list[times][0])] = time_list[times][1]

        #creates dataframe 'data_raw' with the dictionary:
        data_raw = pd.DataFrame(raw_data_dict)

        #transposes the dataframe:
        data_raw = data_raw.transpose()

        #sets the index to the first column:
        data_raw.reset_index(level=0, inplace=True)

        #sets the first row as column-names:
        data_raw.columns = data_raw.iloc[0]

        #deletes the first row
        data_raw = data_raw[1:]

        #displays dataframe 'data_raw':
        #print(data_raw)#[data_raw['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


    #
    #
    elif pd.isna(data_xls.iloc[0, 0]):
        #creates dataframe 'data_raw' from the .csv-file:
        data_raw = data_xls.copy()                                             

        #deletes rows with unnecessary information (temp.-values and measurement-data):
        data_raw = data_raw[:-5]
        data_raw = data_raw.iloc[1:]

        #brings the dataframe in an optimised form for plotting:
        data_raw = data_raw.rename(columns={'Unnamed: 0': 'Well'})

        #deletes the 's' from the time-values:
        for index in range(len(data_raw.columns)):
            data_raw.columns.values[index] =  data_raw.columns.values[index].replace('s', '')

        #add a '0' for Wells with single digit numbers: 
        for index in range(len(data_raw)):
            
            if len(str(data_raw.iloc[index, 0])) == 2:
                
                string = str(data_raw.iloc[index, 0])
                string = string[:1] + '0' + string[1:] 
                data_raw.iloc[index, 0] = string

        #displays dataframe 'data_raw':
        #print(data_raw)#[data_raw['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Wide-to-long-conversion

    #melts dataframe 'data_raw' from wide to long form, where Well is an 'ID', 
    #by which specific values (Time & Value) are identified:
    data_melted = pd.melt(data_raw, id_vars = ['Well'], var_name = 'Time', value_name = 'Value')

    #changes data type of the columns 'Time' & 'Value':
    data_melted['Time'] = data_melted['Time'].astype(float)
    data_melted['Value'] = data_melted['Value'].astype(float)

    #displays dataframe 'data_melted':
    #print(data_melted)#[data_melted['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Curve-Fitting

    #creates list of all wells in dataframe 'data_merged' & deletes duplicates:
    well_plate = list(dict.fromkeys(list(data_melted['Well'])))

    #calls function 'curve_fit_calc' for every well in well_plate & splits the
    #output to two variables:
    fitted_values, popt_list = map(list, zip(*list(curve_fit_calc(data_melted, well) for well in well_plate)))
        
    #displays list 'fitted_values':
    #print(fitted_values)


################################################################################
## Creation of second dataframe

    fitted_data = []

    #primary for-loop over all fitted values:
    for wells in range(len(fitted_values)):
        
        #secondary for-loop over time-values for the actual well: 
        for position in range(len(fitted_values[wells][1])):
            
            #get well-name, time & fitted value
            well = fitted_values[wells][0]
            time = fitted_values[wells][1][position]                                        
            fitted_y = fitted_values[wells][2][position]
            
            fitted_data.append([well, time, fitted_y])

    #creates dataframe 'data_fitted':
    data_fitted = pd.DataFrame(fitted_data, columns = ['Well', 'Time', 'Fitted Value'])

    #displays dataframe 'data_fitted':
    #print(data_fitted)#[data_fitted['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Dataframe-merging

    #create multiindex 'Well' and 'Time' for dataframe 'data_fitted':
    data_fitted_multiindex = data_fitted.set_index(['Well', 'Time'], drop = True)

    #check if dataframe 'data_merged' exist:
    try:
        data_merged
    except NameError:
        data_merged_exists = False
    else:
        data_merged_exists = True

    if data_merged_exists == True: 
        
        #check if dataframe 'data_merged' already a has a column 'µ_max':
        if hasattr(data_merged, 'Fitted Value'):
        
            #if TRUE delete the column and merge 'data_µ_max' into 'data_merged':
            del data_merged['Fitted Value']
            data_merged = data_melted.join(data_fitted_multiindex, on = ['Well', 'Time'])

    #if 'data_merged' doesn´t exist join 'data_µ_max' into 'data_merged':
    else:
        data_merged = data_melted.join(data_fitted_multiindex, on = ['Well', 'Time'])

    #display dataframe 'data_merged':
    #print(data_merged)#[data_merged['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Plotting of raw and fitted data

    #creates copies of dataframe 'data_merged':
    data_merged_plot = data_merged.copy()
    data_merged_plot_copy = data_merged_plot.copy()

    #deletes columns from copied dataframes and adds column 'Source' with 
    #specific value:
    del data_merged_plot['Fitted Value']
    data_merged_plot.loc[:, 'Source'] = 'raw data'

    del data_merged_plot_copy['Value']
    data_merged_plot_copy.loc[:, 'Source'] = 'fitted data'

    #appends copied dataframe to 'data_merged_plot': 
    data_merged_plot = data_merged_plot.append(data_merged_plot_copy, ignore_index=True, sort=False)
    
    #appends number of each well in dataframe 'data_merged' to the list &  
    #deletes duplicates:
    well_numbers = list(dict.fromkeys(list(i[1:] for i in list(data_merged_plot['Well']))))

    #determine number of columns to plot diagrams in:
    plot_columns = int(max(well_numbers))

    #deactivates the max_row limit of altair for the processed dataframe:
    alt.data_transformers.enable('default', max_rows=None)

    #sets a domain and range to define the colors for the subplots:
    domain = ['raw data', 'fitted data']
    range_ = ['teal', 'black']

    #raw-data-plot:
    raw_data = alt.Chart(data_merged_plot).mark_point(opacity = 0.4

    #columns to plot with specifications for the axis:
    ).encode(
        alt.X('Time', scale=alt.Scale(domain=(0, 66000)), axis=alt.Axis(title='Time [s]')),
        alt.Y('Value', axis=alt.Axis(title='OD (600 nm)')),
        color = 'Source'#alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))

    #properties of the resulting diagramm:
    ).properties(
        width = 300,
        height = 200
    )

    #fitted-data-plot:
    fitted_curves = alt.Chart(data_merged_plot).mark_line(
    ).encode(
        alt.X('Time'),
        alt.Y('Fitted Value'),
        color = 'Source'#alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
    )

    plot_legend = alt.Chart(data_merged_plot).mark_circle(size = 0, filled = True, opacity = 1.0
    ).encode(
        alt.X('Time:Q'),
        alt.Y('Value:Q'),
        color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
    )

    #command to layer the raw- and fitted-data-plots over each other and split them into many diagramms depending on the well:
    raw_plot = alt.layer(
          plot_legend,
          raw_data,
          fitted_curves,
          data = data_merged_plot
      ).facet(
          facet = 'Well',
          columns = plot_columns,
      ).resolve_axis(
          x='independent',
          y='independent',
      ).resolve_scale(
          color = 'independent'
      ).configure_legend(
          orient = 'right',
          labelFontSize = 15,
          symbolSize = 25
      ).display()


################################################################################
## End-condition:

    third_block_done = True


################################################################################
## Start-condition not met:
else:
    print('-> Please run the previous block first. <-')


# 4. Compute SGT-method & plot results
Computes the SGT with the resulting log-level-reduction and FICI-value for each well. The results are plotted.

In [ ]:
################################################################################
## Initialize parameters:

#@markdown # Cut-off-value for calculation
#@markdown Change the CutOff-value for the calculation: 
cut_off = 0.6 #@param {type:"number"}

#@markdown # Decision to use a linear area for calculations
#@markdown Check the box if you want to manually set a linear area for the 
#@markdown SGT-calculation:
use_linear_area = False #@param {type:"boolean"}

#@markdown Set upper and lower boundaries for the linear area over all plots
#@markdown (only necessary if the box is checked):
upper_boundary =   0.6 #@param {type:"number"}
lower_boundary =   0.4 #@param {type:"number"}

#@markdown ***Click the play-button to start.***


################################################################################
## Start-condition:

try:
    third_block_done
except NameError:
    third_block_done_exists = False
else:
    third_block_done_exists = True

if third_block_done_exists == True:
    
    
################################################################################
## µ-calculation & tangent-application

    #check if the user want to manually perform the tangent-fitting:
    if use_linear_area == True:

        #gets list of all wells in 'data_merged' & deletes duplicates:
        well_plate = list(dict.fromkeys(list(data_merged['Well'])))

        #calls function 'µ_calc_by_boundaries' for every well in 'well_plate':
        boundary_values = list(µ_calc_by_boundaries(data_merged, well, upper_boundary, lower_boundary) for well in well_plate)

        data_µ_max = pd.DataFrame(boundary_values, columns = ['Well', 'Upper Boundary', 'Time', 'Lower Boundary', 'Lower Time', 'µ_max'])
        data_µ_max = data_µ_max.filter(items = ('Well', 'Time', 'µ_max'), axis=1)

        #create multiindex 'Well' and 'Time' for dataframe 'data_µ_max':
        data_µ_max_multiindex = data_µ_max.set_index(['Well', 'Time'], drop = True)

        #check if dataframe 'data_merged' already a has a column 'µ_max':
        if hasattr(data_merged, 'µ_max'):
        
            #if TRUE delete the column and merge 'data_µ_max' into 
            #'data_merged':
            del data_merged['µ_max']
            data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])

        #if FALSE merge 'data_µ_max' into 'data_merged':
        else:
            data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])
        
        #creates an array of all rows from 'data_merged' that match the wells
        #listed in 'boundary_values' and where 'µ_max' is not 'nan':
        value_array_from_data_merged = list(data_merged.loc[(data_merged['Well'] == elements[0]) & (pd.notna(data_merged['µ_max'])), :].values[0] for elements in boundary_values)
        
        #calls function 'n_calc' for every element in 'value_array_from_data
        #_merged' & splits output into two lists:
        y0_list, addition_for_data_merged = map(list, zip(*list(n_calc(array) for array in value_array_from_data_merged)))
        
        #creates dataframes from the resulting arrays:
        data = pd.DataFrame(y0_list, columns = ['Well', 'µ_max', 'y0'])
        
        #display resulting dataframe:
        #print(data)

    #
    #
    #check if the user want to autmatically perform the tangent-fitting:
    if use_linear_area == False:
        
        #Wgets list of all wells in 'data_merged' & deletes duplicates:
        well_plate = list(dict.fromkeys(list(data_merged['Well'])))

        #calls function 'µ_calc' for every well in 'well_plate':
        µ_values = list(µ_calc(data_merged, well) for well in well_plate)

        #display resulting array:
        #print(µ_values)

        #calls function 'find_µ_max' for every array in 'µ_values':
        µ_max_list = list(find_µ_max(array) for array in µ_values)

        #set Pandas display-precision to show 3 decimal places:
        pd.set_option('display.precision', 3)

        #create a dataframe from the resulting array:
        data_µ_max = pd.DataFrame(µ_max_list, columns = ['Well', 'Time', 'µ_max'])

        #display resulting dataframe:
        #print("data_µ_max:\n", data_µ_max)#[data_µ_max['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed

        #create multiindex 'Well' and 'Time' for dataframe 'data_µ_max':
        data_µ_max_multiindex = data_µ_max.set_index(['Well', 'Time'], drop = True)

        #check if dataframe 'data_merged' already a has a column 'µ_max':
        if hasattr(data_merged, 'µ_max'):
        
            #if TRUE delete the column and merge 'data_µ_max' into 
            #'data_merged':
            del data_merged['µ_max']
            data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])

        #if FALSE merge 'data_µ_max' into 'data_merged':
        else:
            data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])

        #display dataframe 'data_merged':
        #print("data_merged:\n", data_merged, "\n")#[data_merged['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed

        #creates an array of all rows from 'data_merged' that match the wells
        #listed in 'boundary_values' and where 'µ_max' is not 'nan':
        value_array_from_data_merged = list(data_merged.loc[(data_merged['Well'] == elements[0]) & (pd.notna(data_merged['µ_max'])), :].values[0] for elements in µ_max_list)

        #calls function 'n_calc' for every element in 'value_array_from_data
        #_merged' & splits output into two lists:
        y0_list, unneeded_list = map(list, zip(*list(n_calc(elements) for elements in value_array_from_data_merged)))

        #create dataframe from the resulting array:
        data = pd.DataFrame(y0_list, columns = ['Well', 'µ_max', 'y0'])

        #display dataframe 'data':
        #print("data:\n", data)#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'):")] #Optional: only a User-chosen Well is displayed


################################################################################
## SGT-calculation

    #get row-count of dataframe 'data':
    row_count = data.shape[0]

    #calls function 'SGT_calc' for every row in 'row_count':
    SGT_values = list(SGT_calc(data, rows, cut_off) for rows in range(row_count))

    #applies values from 'SGT_values' to dataframe 'data':
    for row in range(row_count):
        
        data.loc[row, 'SGT'] = SGT_values[row]

    #displays dataframe 'data':
    #print(data)#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## log-level-calculation

    #calls function 'board_iteration' for every checkerboard:
    calculated_values = list(board_iteration(data, antibiotic_rows_conc, 
                             antibiotic_columns_conc, checkerboards, boards, 
                             board_count) for boards in range(len(checkerboards)))
    
    #join nested lists to one list:
    calculated_values = [inner for outer in calculated_values for inner in outer]
    
    #creates dataframe from calculated data:
    data_calculated_values = pd.DataFrame(calculated_values, columns = ['Well',
        'SGT minus control', 'log-level-reduction',
        (antibiotic_rows_name + '-concentration'),
        (antibiotic_columns_name + '-concentration')])

    #displays dataframe 'data_calculated_values':
    #print(data_calculated_values)#[data_calculated_values['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Dataframe-merging

    #create index 'Well' for dataframe 'data_calculated_values':
    data_calculated_values_index = data_calculated_values.set_index(['Well'], drop = True)

    first_antibiotic_column = str(antibiotic_rows_name + '-concentration')
    second_antibiotic_column = str(antibiotic_columns_name + '-concentration')

    #check if dataframe 'data' already has the columns from dataframe 'data_calculated_values'
    #if TRUE delete the columns and merge 'data_calculated_values' into 'data':
    if hasattr(data, 'SGT minus control'):
        del data['SGT minus control']
        
        if hasattr(data, 'log-level-reduction'):
            del data['log-level-reduction']
            
            if hasattr(data, first_antibiotic_column):
                del data[first_antibiotic_column]
                
                if hasattr(data, second_antibiotic_column):
                    del data[second_antibiotic_column]
                    data = data.merge(data_calculated_values_index, on = ['Well'])
                
                else:
                    data = data.merge(data_calculated_values_index, on = ['Well'])
            else:
                data = data.merge(data_calculated_values_index, on = ['Well'])
        else:
            data = data.merge(data_calculated_values_index, on = ['Well'])

    #if FALSE merge 'data_calculated_values' into 'data':
    else:
        data = data.merge(data_calculated_values_index, on = ['Well'])

    #display dataframe 'data':
    #print(data)#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Determination of BBC-wells

    #calls function 'BBC_determination_per_board' for every checkerboard &
    #splits output into two lists:
    first_antibiotic_bbcs, second_antibiotic_bbcs = map(list, zip(*list(BBC_determination_per_board(data, checkerboards, boards) for boards in range(len(checkerboards)))))

    #displays both lists:
    #print(first_antibiotic_bbcs, second_antibiotic_bbcs)

    #appends the first element of every nested list in the 'first/second_
    #antibiotic_bbcs'-list to a new list:
    first_antibiotic_first_bbc = list(elements[0] for elements in first_antibiotic_bbcs)
    second_antibiotic_first_bbc = list(elements[0] for elements in second_antibiotic_bbcs)

    #displays new main-arrays:
    #print(first_antibiotic_first_bbc, second_antibiotic_first_bbc)


################################################################################
## Determination of synergism

    #calls function 'fici_values_per_board_calc' for every checkerboard:
    fici_values = list(fici_values_per_board_calc(data, checkerboards, boards, 
                                                  first_antibiotic_first_bbc,
                                                  second_antibiotic_first_bbc,
                                                  first_antibiotic_column,
                                                  second_antibiotic_column
                                                  ) for boards in range(len(checkerboards)))

    #displays main-array:
    #print(fici_values)


################################################################################
## Then from this wells the first of each row and column are identified.

    #get a list of all wells in dataframe 'data':
    plate_wells = data['Well'].values

    #calls function 'first_well_identifier' for each checkerboard:
    first_wells_in_rows_and_columns = list(first_well_identifier(checkerboards,
                                           boards, fici_values) for boards in 
                                           range(len(checkerboards)))

    #displays the resulting array:   
    #print(first_wells_in_rows_and_columns)


################################################################################
## For the first wells of each row and column of the checkerboard the average of 
## the FICI-values is determined, which allows a statement about an possible 
## synergism between the two antibiotics.

    #calls function 'FICI_finder_per_board' for every checkerboard:
    fici_values_with_well_per_board = list(FICI_finder_per_board(boards, fici_values, first_wells_in_rows_and_columns) for boards in range(len(checkerboards)))

    #merges the nested lists in 'fici_values_with_well_per_board':
    fici_values_with_well = list(inner for outer in fici_values_with_well_per_board for inner in outer)

    #creates dataframe 'data_fici_values' from 'fici_values_with_well':
    data_fici_values = pd.DataFrame(fici_values_with_well, columns = ['Well', 'FICI-Value'])

    #calls function 'append_FICI_values_to_board' for every checkerboard:
    board_fici_values = list(append_FICI_values_to_board(fici_values_with_well_per_board, boards) for boards in range(len(checkerboards)))    

    #computes the average of the fici-values of the first wells in all rows and
    #columns of the checkerboard:
    fici_average = list(np.average(board_fici_values[boards]) for boards in range(len(checkerboards)))

    #displays the resulting arrays:
    for boards in range(len(checkerboards)):
        
        print("First wells of all rows and columns of Checkerboard %s with an log-level-reduction >= 3 and their corresponding FICI-value:"
        %(boards + 1), '\n',
        data_fici_values[data_fici_values['Well'].isin(first_wells_in_rows_and_columns[boards])].set_index(['Well'], drop = True),
        '\n\n',
        "The average of the FICI-values for this board is: ",
        fici_average[boards],
        '\n')


################################################################################
## Summarise results in files

    if os.path.exists('result.md') == True:
        os.remove("result.md")

    if os.path.exists('formulas.md') == True:
        os.remove("formulas.md")

    #creates copy of dataframe 'data':
    data_results = data.copy()

    #checks if the new dataframe contains the columns 'µ_max' & 'y0' & deletes
    #them if TRUE:
    if hasattr(data_results, 'µ_max'):
        del data_results['µ_max']
        
        if hasattr(data_results, 'y0'):
            del data_results['y0']

    #reorders the columns of the dataframe:
    data_results_columns = list(data_results.columns)
    new_columns_order = [data_results_columns[0], data_results_columns[-2], data_results_columns[-1]]
    new_columns_order.extend(data_results_columns[1:-2])
    data_results = data_results[new_columns_order]

    checkerboard_dict = {}

    row_count = data_results.shape[0]

    #for-loop over all checkerboards:
    for boards in range(len(checkerboards)):

        #initialize data for if-conditions:
        first_well = str(checkerboards[boards][1]).strip()
        last_well = str(checkerboards[boards][2]).strip()

        well_list = []
        
        #for-loop over all wells in dataframe 'data_results':
        for rows in range(row_count):
            
            #get name of the specific well:
            well_name = str(data_results.iat[rows, 0])
            
            #check if well is within the limits of the checkerboard and 
            #accomplishs the log-level-reduction:
            if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
                if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:

                    well_list.append(well_name)
            
            elif well_name[0] <= first_well[0] or well_name[0] >= last_well[0]:
                if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:
                    
                    well_list.append(well_name)
            
            elif well_name[1:] <= first_well[1:] or well_name[1:] >= last_well[1:]:
                if well_name[0] >= first_well[0] or well_name[0] <= last_well[0]:
                    
                    well_list.append(well_name)
        
        #creates copy of dataframe 'data_results':
        data_results_copy = data_results.copy()

        #gets actual index + 1:
        boards_index = (boards + 1)
        
        #checks if the index is < 10:
        if boards_index < 10:
            
            #if TRUE a 0 is added in front of the index & the so numbered 
            #checkerboard is used as key in the dicitionary which references 
            #the dataframe 'data_results_copy' without all rows whose well 
            #wasn´t in well_list. Sets the column with the well-names as index:
            checkerboard_dict["Checkerboard_0{0}".format(boards_index)] = data_results_copy.drop(data_results_copy[data_results_copy.Well.isin(well_list) == False].index).set_index('Well', drop = True).to_markdown()
        
        elif boards_index >= 10:
            
            #if FALSE the  Checkerboard is just numbered with the index & 
            #used as key in the dicitionary which references the dataframe 
            #'data_results_copy' without all rows whose well wasn´t in 
            #well_list. Sets the column with the well-names as index:
            checkerboard_dict["Checkerboard_{0}".format(boards_index)] = data_results_copy.drop(data_results_copy[data_results_copy.Well.isin(well_list) == False].index).set_index('Well', drop = True).to_markdown()

    #displays the actual key (= Checkerboard-number) & the corresponding 
    #value (= dataframe) of the dictionary:
    for keys in range(len(checkerboard_dict)):
        
        actual_key = list(checkerboard_dict)[keys]
        #print(actual_key, '\n', checkerboard_dict[actual_key], '\n')
        
        #variable that references actual checkerboard from the dictionary
        result_md = checkerboard_dict[actual_key]
        
        if keys == 0:
            
            #creates a file 'result.md' with the command to write data to it:
            result_file = open("Results.md", "w")
        
        elif keys > 0:
            
            #creates a file 'result.md' with the command to append data to it:
            result_file = open("Results.md", "a")
        
        #writes data to the opened file:
        result_file.write('## ')
        result_file.write(actual_key)
        result_file.write('\nThe average FICI-value of the Checkerboard is:  ')
        result_file.write(str(fici_average[keys]))
        result_file.write('\n')
        result_file.write(result_md)
        result_file.write('\n\n')
        
        #closes the file:
        result_file.close()

    #creates dataframe:
    data_sigmoid_parameters = pd.DataFrame(popt_list, columns = ['Well',
        'Sigmoid-function parameters: L', 'Sigmoid-function parameters: x0', 
        'Sigmoid-function parameters: k', 'Sigmoid-function parameters: n'])

    #copies dataframe 'data':
    data_formulas = data.copy()

    #deletes unneeded columns and merges the dataframes:
    del data_formulas['SGT minus control']
    del data_formulas['log-level-reduction']
    del data_formulas['Nitroxolin-concentration']
    del data_formulas['Dalbavancin-concentration']

    if hasattr(data_formulas, 'Sigmoid-function parameters: L'):
        del data_formulas['Sigmoid-function parameters: L']
        
        if hasattr(data, 'Sigmoid-function parameters: x0'):
            del data['Sigmoid-function parameters: x0']
            
            if hasattr(data_formulas, 'Sigmoid-function parameters: k'):
                del data_formulas['Sigmoid-function parameters: k']
                
                if hasattr(data_formulas, 'Sigmoid-function parameters: n'):
                    del data_formulas['Sigmoid-function parameters: n']
                    data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
                
                else:
                    data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
            else:
                data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
        else:
            data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])

    else:
        data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])

    #sets column 'Well' as index & renames columns:
    data_formulas = data_formulas.set_index('Well', drop = True)
    data_formulas = data_formulas.rename(columns={'µ_max' : 'm', 'y0' : 'n'})

    formulas_md = data_formulas.to_markdown()

    sigmoid_formula = 'y = L / (1 + e^(-k * (x - x0))) + n'
    tangent_formula = 'y = m * x + n'

    #writes results to file:
    result_file = open("Results.md", "a")
    result_file.write('## Parameters of the tangent- and the sigmoid-function of each well\n\n')
    result_file.write('Parameters for the formulars of the tangent- and sigmoid-function of each well.\n')
    result_file.write('The tangent-function is defined as:\n')
    result_file.write(tangent_formula)
    result_file.write('\n\n')
    result_file.write('The sigmoid-function is defined as:\n')
    result_file.write(sigmoid_formula)
    result_file.write('\n\n')
    result_file.write(formulas_md)
    result_file.close()


################################################################################
## Plot resulting data

    #get copy of dataframe 'data_merged':
    data_results_plot = data_merged.copy()

    #create lists of all wells & time-values & delete duplicates:
    time_values = list(dict.fromkeys(data_results_plot['Time'].values))
    well_list = list(dict.fromkeys(data_results_plot['Well'].values))

    #calls the function 'tangent_value_calc' for every well in 'well_plate':
    tangent_values = list(tangent_value_calc(data, data_results_plot, well, time_values) for well in well_plate)

    #joins the nested lists in 'tangent_values':
    tangent_values = [inner for outer in tangent_values for inner in outer]

    #creates dataframe from 'tangent_values':
    data_tangent_values = pd.DataFrame(tangent_values, columns = ['Well', 'Time', 'Tangent y-value'])
    
    #creates multiindex on 'Well' & 'Time' for dataframe 'data_tangent_values':
    data_tangent_values_multiindex = data_tangent_values.set_index(['Well', 'Time'], drop = True)

    #checks if 'data_results_plot' already has column 'Tangent y_value':
    if hasattr(data_results_plot, 'Tangent y-value'):
        
        #if TRUE delete the column & merge 'data_tangent_values_multiindex'
        #into 'data_results_plot':
        del data_results_plot['Tangent y-value']
        data_resutls_plot = data_results_plot.merge(data_tangent_values_multiindex, on = ['Well', 'Time'])

    #if FALSE just merge both dataframes:
    else:
        data_results_plot = data_results_plot.merge(data_tangent_values_multiindex, on = ['Well', 'Time'])

    #if linear area is used append the upper & lower boundary y_value to
    #dataframe 'data_results_plot':
    if use_linear_area == True:
        data_results_plot.loc[:, 'Upper Boundary'] = upper_boundary
        data_results_plot.loc[:, 'Lower Boundary'] = lower_boundary

    #get list of all wells in 'data_results_plot':
    plate_wells = list(data_results_plot['Well'])

    #get list of all numbers used in well-names in 'data_results_plot':
    well_numbers = list(dict.fromkeys([wells[1:] for wells in plate_wells]))

    #set number of columns in which subplots shall be displayed:
    plot_columns = int(max(well_numbers))

    #transform dataframe 'data_results_plot' into long-form:
    if use_linear_area == True:
        
        data_results_plot_upper = data_results_plot.filter(items = 
                               ('Well', 'Time', 'Upper Boundary'), axis=1)
        data_results_plot_upper.loc[:, 'Source'] = 'upper boundary'


        data_results_plot_lower = data_results_plot.filter(items = 
                               ('Well', 'Time', 'Lower Boundary'), axis=1)
        data_results_plot_lower.loc[:, 'Source'] = 'lower boundary'


    data_results_plot_fitted = data_results_plot.filter(items = 
                               ('Well', 'Time', 'Fitted Value'), axis=1)
    data_results_plot_fitted.loc[:, 'Source'] = 'fitted data'


    data_results_plot_tangent = data_results_plot.filter(items = 
                               ('Well', 'Time', 'Tangent y-value'), axis=1)
    data_results_plot_tangent.loc[:, 'Source'] = 'tangent'


    data_results_plot_µ_max = data_results_plot.filter(items = 
                               ('Well', 'Time', 'µ_max corresponding Value')
                               , axis=1)
    data_results_plot_µ_max.loc[:, 'Source'] = 'µ_max'


    data_results_plot = data_results_plot.filter(items = 
                        ('Well', 'Time', 'Value'), axis=1)
    data_results_plot.loc[:, 'Source'] = 'raw data'


    if use_linear_area == True:
        
        data_results_plot = data_results_plot.append([data_results_plot_fitted,
                                                      data_results_plot_tangent,
                                                      data_results_plot_µ_max,
                                                      data_results_plot_upper,
                                                      data_results_plot_lower
                                                      ], ignore_index = True, 
                                                      sort = False)
        
    else:
        
        data_results_plot = data_results_plot.append([data_results_plot_fitted,
                                                      data_results_plot_tangent,
                                                      data_results_plot_µ_max
                                                      ], ignore_index=True, 
                                                      sort=False)
    
    
    #
    alt.data_transformers.enable('default', max_rows=None)

    #give colors for specific values:
    if use_linear_area == True:
        
        domain = ['raw data', 'fitted data', 'tangent', 'upper boundary', 'lower boundary']
        range_ = ['teal', 'black', 'orange', 'red', 'red']

    else:
        
        domain = ['raw data', 'fitted data', 'tangent', 'point of µ_max']
        range_ = ['teal', 'black', 'orange', 'red']

    #define altair-charts:
    raw_data = alt.Chart(data_results_plot).mark_point(size = 20, filled = True
        ).encode(
            alt.X('Time:Q', scale=alt.Scale(domain=(0, 66000)), axis=alt.Axis(title='Time [s]')),
            alt.Y('Value:Q', scale=alt.Scale(domain=(0, 1.5)), axis=alt.Axis(title='OD (600 nm)')),
            color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
        ).transform_filter(
            alt.FieldRangePredicate(field='Value', range=[0, 1.5])
        )


    fitted_curves = alt.Chart(data_results_plot).mark_line(
        ).encode(
            alt.X('Time:Q'),
            alt.Y('Fitted Value:Q'),
            color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
        )


    tangent = alt.Chart(data_results_plot).mark_line(opacity = 0.5
        ).encode(
            alt.X('Time:Q'),
            alt.Y('Tangent y-value:Q'),
            color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
        ).transform_filter(
            alt.FieldRangePredicate(field='Tangent y-value', range=[0, 1.5]))

    #check if manually a linear area was set:
    if use_linear_area == True:
        
        #if TRUE initialize plots for upper and lower border:
        upper_border = alt.Chart(data_results_plot).mark_line(opacity = 0.35
            ).encode(
                alt.X('Time:Q'),
                alt.Y('Upper Boundary:Q'),
                color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
            )

        lower_border = alt.Chart(data_results_plot).mark_line(opacity = 0.35
            ).encode(
                alt.X('Time:Q'),
                alt.Y('Lower Boundary:Q'),
                color = alt.Color('Source', scale = alt.Scale(domain = domain, range = range_), legend = alt.Legend(title = ""))
            )

        #define plot from subplots:
        plotted = alt.layer(
                raw_data,
                fitted_curves,
                tangent,
                upper_border,
                lower_border,
                data = data_results_plot
            ).facet(
                facet = 'Well',
                columns = plot_columns
            ).resolve_axis(
                x='independent',
                y='independent'
            ).resolve_scale(
                color = 'independent'
            ).configure_legend(
                orient = 'right',
                labelFontSize = 15,
                symbolSize = 20
            )

    else:

        #initialize plot for µ_max-point:
        µ_max_point = alt.Chart(data_results_plot).mark_point(color= 'red', size = 75, filled = True
        ).encode(
                alt.X('Time:Q'),
                alt.Y('µ_max corresponding Value:Q')
            )
        

        #define plot from subplots:
        plotted = alt.layer(
                raw_data,
                fitted_curves,
                tangent,
                µ_max_point,
                data = data_results_plot
            ).facet(
                facet = 'Well',
                columns = plot_columns
            ).resolve_axis(
                x='independent',
                y='independent'
            ).resolve_scale(
                color = 'independent'
            ).configure_legend(
                orient = 'right',
                labelFontSize = 15,
                symbolSize = 20,
            )

    #plot and save plot:
    plotted.display()
    plotted.save("chart.svg", scale_factor = 5.0)


################################################################################
## End-condition:

    fourth_block_done = True


################################################################################
## Start-condition not met:

else:
    print('-> Please run the previous block first. <-')

# 5. Download

In [ ]:
#@markdown ***Click the Play-button to download the results.***


################################################################################
## Start-condition:

try:
    fourth_block_done
except NameError:
    fourth_block_done_exists = False
else:
    fourth_block_done_exists = True

if fourth_block_done_exists == True:


################################################################################
## Download:

    from google.colab import files
    files.download('Results.md')
    files.download('chart.svg')


################################################################################
## Start-condition not met:

else:
    print('-> Please run the previous block first. <-')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>